# Importing modules

In [2]:
import pandas as pd
import os
import hvplot.pandas
import panel as pn

# Reading data and removing unnecessary columns
This will read the most recent CSV file in the Downloads folder and import it as a dataframe. 

In [3]:
download_directory = "C:\\Users\\mtare\\Downloads"

recent_file = ["", 0]
for file in os.listdir(download_directory):
    if file.endswith(".csv") and "attendance_reports_attendance" in file:
        file_directory = f"{download_directory}\\{file}"
        
        if os.path.getctime(file_directory)> recent_file[1]:
            recent_file = [file_directory, os.path.getctime(file_directory)]
            
df = pd.read_csv(recent_file[0], index_col=False)
df = df.drop(columns=["Course ID", "SIS Course ID", "Course Code","Section Name", "Section ID", "SIS Section ID", "Teacher ID", "Teacher Name", "Timestamp"], axis = 1)

course_name = df["Course Name"].loc[0]

df.head()


,Course Name,Student ID,Student Name,Class Date,Attendance
0,MATHXXX,453142,Ky10Sm,8/20/2024,present
1,MATHXXX,327247,Kr15Ta,8/20/2024,present
2,MATHXXX,479285,Ki16Ag,8/20/2024,present
3,MATHXXX,713724,Ce10Lu,8/20/2024,present
4,MATHXXX,595385,Al16Ab,8/20/2024,present


# Creating new, readable Dataframe with absent counter

In [5]:
# generating list of unique names and dates
names_list = []
dates_list = []
for i in range(len(df)):
    name = df["Student Name"].loc[i]
    date = df["Class Date"].loc[i]
    if name not in names_list:
        names_list.append(name)
    if date not in dates_list:
        dates_list.append(date)

names_list.sort()

# creating columns
df_columns = ["Full_Name", "Num_of_absences"]

for date in dates_list:
    df_columns.append(date)


# creating empty dataframe with columns
att_df = pd.DataFrame(columns = df_columns)

# filling dataframe with attendance and counting absences
i = 0
for student in names_list:
    row_data = [student]  # this list will be build with student, num of absences, and row_records
    
    row_records = [] 
    absent_counter = 0
    for date in dates_list:
        
        condition = (df["Student Name"] == student) & (df["Class Date"] == date)
        
        if df[condition].Attendance.empty:  # if attendance was not recorded on a particular date for a student
            row_records.append("n/a")
            absent_counter = absent_counter + 1
            
        else:
            for value in df[condition].Attendance:
                if value != "present":
                    absent_counter = absent_counter + 1
                row_records.append(value)
                
    row_data.append(absent_counter)  # adding absent counter
    row_data.extend(row_records)  # extending row_data to include row_records

  

    att_df.loc[i] = row_data  # adding row of data to dataframe

    i = i + 1

att_df

,Full_Name,Num_of_absences,8/20/2024,8/22/2024,8/26/2024,9/3/2024,9/12/2024,9/17/2024,9/19/2024,9/24/2024,8/29/2024,8/27/2024,10/1/2024,9/26/2024,9/10/2024,9/5/2024
0,Ab13Ma,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present
1,Ai11Ba,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present
2,Al13Go,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present
3,Al14Lo,1,present,present,present,present,present,present,present,absent,present,present,present,present,present,present
4,Al14Sa,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present
5,Al16Ab,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present
6,Al18Ma,1,present,present,present,present,present,absent,present,present,present,present,present,present,present,present
7,Br12Me,12,absent,absent,absent,absent,absent,absent,absent,absent,present,present,absent,absent,absent,absent
8,Br24Mo,1,n/a,present,present,present,present,present,present,present,present,present,present,present,present,present
9,Ce10Lu,0,present,present,present,present,present,present,present,present,present,present,present,present,present,present


# Creating dashboard

In [6]:
table = att_df.hvplot.table(columns=df_columns, sortable=True, selectable=True, height=700)
table

:Table   [Full_Name,Num_of_absences,8/20/2024,8/22/2024,8/26/2024,9/3/2024,9/12/2024,9/17/2024,9/19/2024,9/24/2024,8/29/2024,8/27/2024,10/1/2024,9/26/2024,9/10/2024,9/5/2024]

In [7]:
# creating empty dataframe with columns
count_df = pd.DataFrame(columns = ["Date", "Students_Present"])

present_count = []  # present count by day

i = 1
for day in dates_list:
    row_data = []
    condition = (att_df[day] == "present")
    num_of_students = len(att_df[condition])
    row_data = [day, num_of_students]
    count_df.loc[i] = row_data  # adding row of data to dataframe
    i = i + 1
count_df

,Date,Students_Present
1,8/20/2024,28
2,8/22/2024,29
3,8/26/2024,27
4,9/3/2024,25
5,9/12/2024,26
6,9/17/2024,23
7,9/19/2024,24
8,9/24/2024,22
9,8/29/2024,33
10,8/27/2024,33


In [8]:
bargraph  = count_df.hvplot.barh(y="Students_Present", x="Date",
                                 responsive=True,
                                 min_height=300,
                                 min_width=300,
                                 
                                 max_height=500,
                                 max_width=500,
                                 grid=True,
                                 color = "#9989e8"

                                )
bargraph


:Bars   [Date]   (Students_Present)

In [9]:
template = pn.template.EditableTemplate(
    editable=True,
    title=f"Attendance: {course_name}"
)
template.main.extend(
                    pn.Column(
                        pn.Row(table, bargraph)
                    )
)
template.show()

Launching server at http://localhost:49534
